# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770726326598                   -0.53    8.0         
  2   -2.772127327928       -2.85       -1.30    1.0    137ms
  3   -2.772168910301       -4.38       -2.74    1.0    174ms
  4   -2.772170708765       -5.75       -3.55    2.0    160ms
  5   -2.772170720295       -7.94       -3.85    2.0    163ms
  6   -2.772170722971       -8.57       -5.23    1.0    185ms
  7   -2.772170723014      -10.36       -5.59    2.0    166ms
  8   -2.772170723015      -12.28       -5.87    2.0    170ms
  9   -2.772170723015      -12.94       -6.65    1.0    178ms
 10   -2.772170723015   +  -13.92       -7.28    1.0    161ms
 11   -2.772170723015      -14.51       -8.46    2.0    186ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770732425501                   -0.52    9.0         
  2   -2.7

1.7735578353749593

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770893892660                   -0.53    9.0         
  2   -2.772063786909       -2.93       -1.32    1.0    137ms
  3   -2.772083205475       -4.71       -2.50    1.0    140ms
  4   -2.772083362529       -6.80       -3.23    1.0    145ms
  5   -2.772083415677       -7.27       -3.86    2.0    395ms
  6   -2.772083417686       -8.70       -4.64    1.0    199ms
  7   -2.772083417810       -9.90       -5.42    2.0    221ms
  8   -2.772083417810      -12.58       -6.08    1.0    574ms
  9   -2.772083417811      -12.64       -7.05    2.0    185ms
 10   -2.772083417811   +  -14.07       -7.57    2.0    183ms
 11   -2.772083417811      -14.21       -8.40    1.0    166ms

Polarizability via ForwardDiff:       1.7725349778876651
Polarizability via finite difference: 1.7735578353749593
